<a href="https://colab.research.google.com/github/Jvnc0503/DL-Lab1-Animal-Sounds/blob/main/Laboratorio_1_Animal_Sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = "/content/drive/MyDrive/Animal Sounds/"
local_path = "/content/data/"

!mkdir -p {local_path}

!cp "{drive_path}train.7z" {local_path}
!cp "{drive_path}test.7z" {local_path}
!cp "{drive_path}train.csv" {local_path}

In [ ]:
!apt-get install p7zip-full
!7z x /content/data/train.7z -o{local_path}
!7z x /content/data/test.7z -o{local_path}

In [ ]:
import pandas as pd

train_y = pd.read_csv(f"{local_path}train.csv")
train_y.head()

In [ ]:
!ls /content/data/train/train/ | head -n 10

In [ ]:
import soundfile as sf

file_path = f"{local_path}/train/INCT17_20191113_040000_0_3.wav"
data = sf.info(file_path)

samplerate = data.samplerate

print(f"Sampling Rate exacto: {data.samplerate} Hz")
print(f"Canales: {data.channels}")
print(f"Duración: {data.duration} segundos")
print(f"Formato: {data.format_info} - {data.subtype_info}")

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt

def generate_spectrogram(audio_path: str, image_path: str) -> None:
  y, sr = librosa.load(audio_path)
  n_fft = 2048
  hop_length = 512
  n_mels = 128

  spectrogram = librosa.feature.melspectrogram(
      y=y,
      sr=sr,
      n_fft=n_fft,
      hop_length=hop_length,
      n_mels=n_mels,
      fmin=50,
      fmax=samplerate/2
  )

  spectogram_pcen = librosa.pcen(
      spectrogram * (2**31),
      time_constant=0.06,
      gain=0.8,
      bias=10,
      power=0.25,
      eps=1e-6
  )

  plt.figure(figsize=(10, 4))
  plt.imshow(spectogram_pcen, aspect='auto', origin='lower', cmap='magma')
  plt.axis('off')
  plt.savefig(image_path, bbox_inches='tight', pad_inches=0, dpi=300)
  plt.close()

In [ ]:
import os
from pathlib import Path
from tqdm import tqdm

train_dir = Path(local_path) / "train"
test_dir = Path(local_path) / "test"
output_dir = Path(local_path) / "spectograms"

output_dir.mkdir(parents=True, exist_ok=True)

wav_files = list(train_dir.glob("**/*.wav"), test_dir.glob("**/*.wav"))
print(f"Total de archivos WAV: {len(wav_files)}")

for wav_path in tqdm(wav_files, desc="Generando espectogramas"):
  try:
    save_name = wav_path.with_suffix(".png").name
    save_path = output_dir / save_name

    if save_path.exists():
      continue

    generate_spectrogram(str(wav_path), str(save_path))
  except Exception as e:
    print(f"\nError procesando {wav_path.name}: {e}")

print("¡Procesamiento completado!")